# Задание 3

**Создаем сессию с заданным конфигом**

In [1]:
import org.apache.spark.sql.{DataFrame, SparkSession, Row}

//Пример 1 запуска
val spark = SparkSession.builder
        //.config("spark.jars", "/home/.../spark/postgresql-42.2.20.jar")
        .config("spark.driver.memory", "2g")
        .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-SVGHN8C:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1670447623850)
SparkSession available as 'spark'


22/12/08 00:13:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


import org.apache.spark.sql.{DataFrame, SparkSession, Row}
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3fe2e390


22/12/08 00:13:55 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped


-------------------------------------------------------
**Читаем паркеты с данными**

In [2]:
val employees = spark.read.parquet("D:\\BD_MEPHI\\lab1\\data\\employees.parquet")
val countries = spark.read.parquet("D:\\BD_MEPHI\\lab1\\data\\countries.parquet")
val departments = spark.read.parquet("D:\\BD_MEPHI\\lab1\\data\\department.parquet")
val jobs = spark.read.parquet("D:\\BD_MEPHI\\lab1\\data\\jobs.parquet")
val job_history = spark.read.parquet("D:\\BD_MEPHI\\lab1\\data\\job_history.parquet")
val locations = spark.read.parquet("D:\\BD_MEPHI\\lab1\\data\\locations.parquet")
val regions = spark.read.parquet("D:\\BD_MEPHI\\lab1\\data\\regions.parquet")

employees: org.apache.spark.sql.DataFrame = [employee_id: int, first_name: string ... 9 more fields]
countries: org.apache.spark.sql.DataFrame = [country_id: string, country_name: string ... 1 more field]
departments: org.apache.spark.sql.DataFrame = [department_id: int, department_name: string ... 2 more fields]
jobs: org.apache.spark.sql.DataFrame = [job_id: string, job_title: string ... 2 more fields]
job_history: org.apache.spark.sql.DataFrame = [employee_id: int, start_date: string ... 3 more fields]
locations: org.apache.spark.sql.DataFrame = [location_id: int, street_address: string ... 4 more fields]
regions: org.apache.spark.sql.DataFrame = [region_id: int, region_name: string]


In [17]:
employees.show(5)

+-----------+----------+---------+--------+------------+----------+-------+-------+--------------+----------+-------------+
|employee_id|first_name|last_name|   email|phone_number| hire_date| job_id| salary|commission_pct|manager_id|department_id|
+-----------+----------+---------+--------+------------+----------+-------+-------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|17.06.2003|AD_PRES|24000.0|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|21.09.2005|  AD_VP|17000.0|          null|       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|13.01.2001|  AD_VP|17000.0|          null|       100|           90|
|        103| Alexander|   Hunold| AHUNOLD|590.423.4567|03.01.2006|IT_PROG| 9000.0|          null|       102|           60|
|        104|     Bruce|    Ernst|  BERNST|590.423.4568|21.05.2007|IT_PROG| 6000.0|          null|       103|           60|
+-------

--------------------------------------------------------------
**Импортируем Window, создаем строковую переменную с датой** <br>
(с типом Date адекватно сделать не получилось, поэтому далее костыли через unix timestamp)<br>

**Inner Join'им необходимые DF. Inner, а не Left, потому что в задании нужно вывести данные по городам, а null - это не город** <br>

**В итоге из employees получаем employees_extended - DF, где собраны только нужные нам столбцы** <br>

**Далее уже на нем используем оконную функцию. Используем сортировку по убыванию на столбце total. Нумеруем строки, чтобы потом фильтром взять только первую из каждого города (ту, где total больше всего среди всех строк, относящихся к этому городу)**<br>

**Выводим результат**

In [157]:
import org.apache.spark.sql.expressions.Window

var date = "2015-01-01"

val employees_extended = employees
    .join(departments, departments("department_id") === employees("department_id"), "inner")
    .join(locations, locations("location_id") === departments("location_id"), "inner")
    .withColumn("set_date_string", lit(date))
    .withColumn("set_date", to_date(from_unixtime(unix_timestamp(col("set_date_string"), "yyyy-mm-dd"), "yyyy-mm-dd")))
    .withColumn("hire_date_casted", to_date(from_unixtime(unix_timestamp(col("hire_date"), "dd.mm.yyyy"), "yyyy-mm-dd")))
    .withColumn("months_diff", floor(months_between(col("set_date"), col("hire_date_casted"))))
    .withColumn("total", col("salary") * col("months_diff"))
    .select(
        col("city"),
        col("employee_id"),
        col("hire_date_casted"),
        col("months_diff"),
        col("salary"),
        col("total")
    )

//employees_extended.show(10)

val window  = Window.partitionBy("city").orderBy(col("total").desc)

employees_extended
    .withColumn("row", row_number.over(window))
    .filter(col("row") === 1)
    .select(
        col("city"),
        col("employee_id"),
        col("hire_date_casted"),
        col("months_diff"),
        col("salary"),
        col("total")
    )
    .show()



+-------------------+-----------+----------------+-----------+-------+---------+
|               city|employee_id|hire_date_casted|months_diff| salary|    total|
+-------------------+-----------+----------------+-----------+-------+---------+
|             London|        203|      2002-06-07|        150| 6500.0| 975000.0|
|             Munich|        204|      2002-06-07|        150|10000.0|1500000.0|
|             Oxford|        145|      2004-10-01|        123|14000.0|1722000.0|
|            Seattle|        100|      2003-06-17|        138|24000.0|3312000.0|
|South San Francisco|        122|      2003-05-01|        140| 7900.0|1106000.0|
|          Southlake|        103|      2006-01-03|        107| 9000.0| 963000.0|
|            Toronto|        201|      2004-02-17|        130|13000.0|1690000.0|
+-------------------+-----------+----------------+-----------+-------+---------+



import org.apache.spark.sql.expressions.Window
date: String = 2015-01-01
employees_extended: org.apache.spark.sql.DataFrame = [city: string, employee_id: int ... 4 more fields]
window: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@97ecad
